<a href="https://colab.research.google.com/github/Alerza/nlp_task/blob/main/Get_fasttext_and_llama_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext
!pip install transformers

In [4]:
import pandas as pd
llama_vocab = pd.read_csv('/content/drive/MyDrive/DataSet/vocab7b.csv')
llama_vocab.rename(columns = {'Unnamed: 0':'llamaids'}, inplace = True)

In [5]:
llama_vocab

,llamaids,token
0,0,<unk>
1,1,<s>
2,2,</s>
3,3,<0x00>
4,4,<0x01>
...,...,...
31995,31995,黃
31996,31996,왕
31997,31997,收
31998,31998,弘


In [51]:
def remve_under(token):

    cleantokne = str(token).replace('_','')

    return cleantokne

#sample['tokenizer'] = sample['token'].apply(tokne2tokenize)
llama_vocab['clean'] = llama_vocab['token'].apply(remve_under)
llama_vocab

,llamaids,token,clean
0,0,<unk>,<unk>
1,1,<s>,<s>
2,2,</s>,</s>
3,3,<0x00>,<0x00>
4,4,<0x01>,<0x01>
...,...,...,...
31995,31995,黃,黃
31996,31996,왕,왕
31997,31997,收,收
31998,31998,弘,弘


In [6]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-fa-vectors", filename="model.bin")
model_fa = fasttext.load_model(model_path)

In [24]:
model_path = hf_hub_download(repo_id="facebook/fasttext-en-vectors", filename="model.bin")
model_en = fasttext.load_model(model_path)

In [ ]:
words_fa = model_fa.get_words()
words_fa_df = pd.DataFrame(words_fa, columns=['token'])
words_fa_df['ftindex'] = words_fa_df.index


In [9]:
vocab_fa_merge = pd.merge(llama_vocab,words_fa_df, how='inner', on='token')
vocab_fa_merge = vocab_fa_merge[['llamaids','ftindex','token']]
vocab_fa_merge

,llamaids,ftindex,token
0,2,4,</s>
1,261,74813,er
2,262,481,in
3,264,11708,en
4,265,1973,on
...,...,...,...
8062,31991,1765665,げ
8063,31992,1776654,べ
8064,31993,1557475,边
8065,31995,833169,黃


In [ ]:
words_en = model_en.get_words()
words_en_df = pd.DataFrame(words_en, columns=['token'])
words_en_df['ftindex'] = words_en_df.index
words_en_df

In [70]:
vocab_en_merge = pd.merge(llama_vocab,words_en_df, how='inner', on='token')
vocab_en_merge = vocab_en_merge[['llamaids','ftindex','token']]
vocab_en_merge

,llamaids,ftindex,token
0,2,7,</s>
1,261,10712,er
2,262,8,in
3,264,1931,en
4,265,17,on
...,...,...,...
10302,31979,1414023,才
10303,31981,674572,止
10304,31983,1045506,ɯ
10305,31987,1476715,역


In [10]:
import torch
llamaemmbding = torch.load('/content/drive/MyDrive/DataSet/embeding_tensor_7b_llama.pt')

In [11]:
import numpy as np
model = model_fa

def tokne2fasttext(token):
    ids = model.get_word_id(token)
    if ids>= 0 :
        fasttextID = ids
        fasttextEMBD= model.get_input_vector(ids)
        done = 'direct'
    else:
        ctoken = token.replace('▁','')
        ids = model.get_word_id(ctoken)
        if ids>= 0 :
            fasttextID = ids
            fasttextEMBD= model.get_input_vector(ids)
            done = 'rm_'
        else:
            fasttextID = ids
            fasttextEMBD= model[token]
            done = 'NotAvlbe'
    return fasttextID , fasttextEMBD , done

def tokne2llamaEMBD(llamaids):
    llamaEMBD= llamaemmbding[llamaids].detach().numpy()
    return   llamaEMBD



vocab_fa_merge[ 'llamaEMBD'] = vocab_fa_merge['llamaids'].apply(tokne2llamaEMBD)

#vocab_en_merge[ 'fasttextEMBD'] = vocab_en_merge['ftindex'].apply(tokne2fasttextEMBD)
vocab_fa_merge[['fasttextIDs','fasttextEMBD', 'ftdone']] = vocab_fa_merge['token'].apply(tokne2fasttext).apply(pd.Series)
vocab_fa_merge

,llamaids,ftindex,token,llamaEMBD,fasttextIDs,fasttextEMBD,ftdone
0,2,4,</s>,"[0.001743, 0.001763, -0.014465, -0.01499, 0.00...",4,"[-0.24955219, 0.0291843, 0.015046162, -0.04459...",direct
1,261,74813,er,"[-0.002476, 0.0135, 0.01287, 0.01364, -0.01028...",74813,"[0.1887271, 0.2228782, 0.4353074, 0.19322903, ...",direct
2,262,481,in,"[-0.0119, -0.02966, 0.02472, -0.00782, 0.01378...",481,"[0.09666715, -0.09635353, 0.0984944, -0.181347...",direct
3,264,11708,en,"[-0.001209, 0.0001951, -0.02249, 0.01139, -0.0...",11708,"[-0.103974804, -0.097972944, -0.097109385, 0.0...",direct
4,265,1973,on,"[0.005417, -0.02298, 0.02185, 0.009415, -0.020...",1973,"[0.20756036, -0.04686525, -0.07714969, -0.0846...",direct
...,...,...,...,...,...,...,...
8062,31991,1765665,げ,"[-0.00689, -0.01697, 0.003262, 0.00689, 0.0108...",1765665,"[-0.04260341, -0.094983116, 0.07212939, -0.007...",direct
8063,31992,1776654,べ,"[0.01547, -0.00752, -0.008675, 0.00843, 0.0067...",1776654,"[-0.055301122, -0.11099341, 0.0508824, 0.00604...",direct
8064,31993,1557475,边,"[-0.003428, -0.011086, 0.007187, -0.01665, -0....",1557475,"[-0.043888465, -0.07582598, 0.06649552, 0.0150...",direct
8065,31995,833169,黃,"[0.02136, 0.007473, 0.012344, -0.008415, 0.006...",833169,"[-0.06865658, -0.11658596, 0.11669258, -0.0325...",direct


In [15]:
vocab_fa_merge.to_csv('/content/drive/MyDrive/DataSet/fasttext/merge_llama_fafasttext.csv')

In [82]:
word = vocab_en_merge.iloc[100]
word

llamaids                                                      449
ftindex                                                        61
token                                                         out
fasttextEMBD    [-0.09795454, -0.40460548, 0.015043235, 0.4757...
llamaEMBD       [0.001488, -0.03818, -0.011375, -0.001267, -0....
Name: 100, dtype: object

In [ ]:
model_en.get_nearest_neighbors(word['token'], k=30)

In [ ]:
model_fa.get_word_vector('out')